In [34]:
#Import required libraries

import requests
import zipfile
import io

import pandas as pd

import numpy as np
import scipy.stats as ss

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt




In [35]:
# URL of the zip archive
url = 'https://maven-datasets.s3.amazonaws.com/UK+Train+Rides/UK+Train+Rides.zip'



# The file from the archive which required download
csv_filename = 'railway.csv'

#Download the zip file and check the status
response = requests.get(url)
response.raise_for_status()




In [36]:
#Load file into a ZipFile object
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    print(z.namelist())
    
    #Read the required CSV file 
    with z.open(csv_filename) as f:
        df = pd.read_csv(f)   
   
    

['railway.csv', 'railway_data_dictionary.csv']


In [37]:
        
df.head()

,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,Arrival Destination,Date of Journey,Departure Time,Arrival Time,Actual Arrival Time,Journey Status,Reason for Delay,Refund Request
0,da8a6ba8-b3dc-4677-b176,2023-12-08,12:41:11,Online,Contactless,Adult,Standard,Advance,43,London Paddington,Liverpool Lime Street,2024-01-01,11:00:00,13:30:00,13:30:00,On Time,NaN,No
1,b0cdd1b0-f214-4197-be53,2023-12-16,11:23:01,Station,Credit Card,Adult,Standard,Advance,23,London Kings Cross,York,2024-01-01,09:45:00,11:35:00,11:40:00,Delayed,Signal Failure,No
2,f3ba7a96-f713-40d9-9629,2023-12-19,19:51:27,Online,Credit Card,None,Standard,Advance,3,Liverpool Lime Street,Manchester Piccadilly,2024-01-02,18:15:00,18:45:00,18:45:00,On Time,NaN,No
3,b2471f11-4fe7-4c87-8ab4,2023-12-20,23:00:36,Station,Credit Card,None,Standard,Advance,13,London Paddington,Reading,2024-01-01,21:30:00,22:30:00,22:30:00,On Time,NaN,No
4,2be00b45-0762-485e-a7a3,2023-12-27,18:22:56,Online,Contactless,None,Standard,Advance,76,Liverpool Lime Street,London Euston,2024-01-01,16:45:00,19:00:00,19:00:00,On Time,NaN,No


In [38]:
#Check types of the columns
df.dtypes

Transaction ID         object
Date of Purchase       object
Time of Purchase       object
Purchase Type          object
Payment Method         object
Railcard               object
Ticket Class           object
Ticket Type            object
Price                   int64
Departure Station      object
Arrival Destination    object
Date of Journey        object
Departure Time         object
Arrival Time           object
Actual Arrival Time    object
Journey Status         object
Reason for Delay       object
Refund Request         object
dtype: object

In [39]:
#Check for null values
df.isna().sum()

Transaction ID             0
Date of Purchase           0
Time of Purchase           0
Purchase Type              0
Payment Method             0
Railcard                   0
Ticket Class               0
Ticket Type                0
Price                      0
Departure Station          0
Arrival Destination        0
Date of Journey            0
Departure Time             0
Arrival Time               0
Actual Arrival Time     1880
Journey Status             0
Reason for Delay       27481
Refund Request             0
dtype: int64

In [40]:
#Create columns with departure and arrival date and time
df ['Departure Date and Time'] = df['Date of Journey'] +' '+ df['Departure Time']
df ['Arrival Date and Time'] = df['Date of Journey'] +' '+ df['Arrival Time']
df ['Actual Arrival Date and Time'] = df['Date of Journey'] +' '+ df['Actual Arrival Time']

In [41]:
#All the cities in the dataset have one railways stations, except London with 4 stations.
#To make the visualisation simplier, get rid of excessive names of stations in all the cities except London.

def find_word(elem):
    if not elem.startswith('London'):
        return elem.split(' ')[0]
    else:
        return elem
        
        
    
        
df['Departure Station'] = df['Departure Station'].apply(find_word)
df['Arrival Destination'] = df['Arrival Destination'].apply(find_word)

In [42]:
#Create column 'Route', which will include the name of departure and destination stations
df['Route'] = df['Departure Station'] + '-' + df['Arrival Destination']

In [43]:
#check the dataset again
df.head()


,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,...,Departure Time,Arrival Time,Actual Arrival Time,Journey Status,Reason for Delay,Refund Request,Departure Date and Time,Arrival Date and Time,Actual Arrival Date and Time,Route
0,da8a6ba8-b3dc-4677-b176,2023-12-08,12:41:11,Online,Contactless,Adult,Standard,Advance,43,London Paddington,...,11:00:00,13:30:00,13:30:00,On Time,NaN,No,2024-01-01 11:00:00,2024-01-01 13:30:00,2024-01-01 13:30:00,London Paddington-Liverpool
1,b0cdd1b0-f214-4197-be53,2023-12-16,11:23:01,Station,Credit Card,Adult,Standard,Advance,23,London Kings Cross,...,09:45:00,11:35:00,11:40:00,Delayed,Signal Failure,No,2024-01-01 09:45:00,2024-01-01 11:35:00,2024-01-01 11:40:00,London Kings Cross-York
2,f3ba7a96-f713-40d9-9629,2023-12-19,19:51:27,Online,Credit Card,None,Standard,Advance,3,Liverpool,...,18:15:00,18:45:00,18:45:00,On Time,NaN,No,2024-01-02 18:15:00,2024-01-02 18:45:00,2024-01-02 18:45:00,Liverpool-Manchester
3,b2471f11-4fe7-4c87-8ab4,2023-12-20,23:00:36,Station,Credit Card,None,Standard,Advance,13,London Paddington,...,21:30:00,22:30:00,22:30:00,On Time,NaN,No,2024-01-01 21:30:00,2024-01-01 22:30:00,2024-01-01 22:30:00,London Paddington-Reading
4,2be00b45-0762-485e-a7a3,2023-12-27,18:22:56,Online,Contactless,None,Standard,Advance,76,Liverpool,...,16:45:00,19:00:00,19:00:00,On Time,NaN,No,2024-01-01 16:45:00,2024-01-01 19:00:00,2024-01-01 19:00:00,Liverpool-London Euston


In [45]:
#Change type of selected columns

df[['Date of Purchase', 
    'Date of Journey', 
    'Departure Date and Time',
    'Arrival Date and Time',
    'Actual Arrival Date and Time' ]]\
= df[['Date of Purchase',
      'Date of Journey',
     'Departure Date and Time',
      'Arrival Date and Time',
      'Actual Arrival Date and Time']].apply(pd.to_datetime)

In [46]:
#Create column 'Delay', which will show delay in minutes
df['Delay'] = (df['Actual Arrival Date and Time'] - df['Arrival Date and Time']).dt.total_seconds() // 60

In [47]:
df.head()

,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,...,Arrival Time,Actual Arrival Time,Journey Status,Reason for Delay,Refund Request,Departure Date and Time,Arrival Date and Time,Actual Arrival Date and Time,Route,Delay
0,da8a6ba8-b3dc-4677-b176,2023-12-08,12:41:11,Online,Contactless,Adult,Standard,Advance,43,London Paddington,...,13:30:00,13:30:00,On Time,NaN,No,2024-01-01 11:00:00,2024-01-01 13:30:00,2024-01-01 13:30:00,London Paddington-Liverpool,0.0
1,b0cdd1b0-f214-4197-be53,2023-12-16,11:23:01,Station,Credit Card,Adult,Standard,Advance,23,London Kings Cross,...,11:35:00,11:40:00,Delayed,Signal Failure,No,2024-01-01 09:45:00,2024-01-01 11:35:00,2024-01-01 11:40:00,London Kings Cross-York,5.0
2,f3ba7a96-f713-40d9-9629,2023-12-19,19:51:27,Online,Credit Card,None,Standard,Advance,3,Liverpool,...,18:45:00,18:45:00,On Time,NaN,No,2024-01-02 18:15:00,2024-01-02 18:45:00,2024-01-02 18:45:00,Liverpool-Manchester,0.0
3,b2471f11-4fe7-4c87-8ab4,2023-12-20,23:00:36,Station,Credit Card,None,Standard,Advance,13,London Paddington,...,22:30:00,22:30:00,On Time,NaN,No,2024-01-01 21:30:00,2024-01-01 22:30:00,2024-01-01 22:30:00,London Paddington-Reading,0.0
4,2be00b45-0762-485e-a7a3,2023-12-27,18:22:56,Online,Contactless,None,Standard,Advance,76,Liverpool,...,19:00:00,19:00:00,On Time,NaN,No,2024-01-01 16:45:00,2024-01-01 19:00:00,2024-01-01 19:00:00,Liverpool-London Euston,0.0


In [48]:
#Save updated dateset in csv for further operation in Tableau
df.to_csv('.......')